In [1]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [2]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [3]:
params = {"grupo":"10"}

In [4]:
r = requests.get(base_url, params=params)

In [5]:
page = BeautifulSoup(r.text, 'html.parser')

In [6]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [7]:
__EVENTVALIDATION

'/wEdAAy3A5BrTqdQW7Ts7AUWfgtT4mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc3rX1c6SCEekGomAC95lda57M1mGA=='

In [8]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [9]:
__VIEWSTATE

'/wEPDwUJMzc0Mzc1MzA2DxYGHgtzdGF0ZV9ncnVwbwUCMTAeCGlkZkdydXBvAgoeCGJ1c2NhUG9yZBYCZg9kFgICAw9kFgICAQ8WAh4GYWN0aW9uBR8vZGFkb3NhYmVydG9zL2NvbnN1bHRhP2dydXBvPTEwFg4CAg8QDxYGHg1EYXRhVGV4dEZpZWxkBQpOb21lUGFnaW5hHg5EYXRhVmFsdWVGaWVsZAUJVXJsUGFnaW5hHgtfIURhdGFCb3VuZGcWBB4Jb25rZXlkb3duBUNpZihJc0VudGVyS2V5KGV2ZW50KSl7UmVkaXJlY2lvbmFyQWNlc3NvUmFwaWRvKHRoaXMpO3JldHVybiBmYWxzZTt9HgdvbmNsaWNrBR9SZWRpcmVjaW9uYXJBY2Vzc29SYXBpZG8odGhpcyk7EBUGDkFjZXNzbyBSw6FwaWRvFlNlY3JldGFyaWFzIGUgw5NyZ8Ojb3MRU2VydmnDp29zIE9uLUxpbmUYU2VydmnDp29zIHBhcmEgYSBFbXByZXNhGVNlcnZpw6dvcyBwYXJhIG8gQ2lkYWTDo28TU2l0ZXMgZGEgUHJlZmVpdHVyYRUGAA0vc2VjcmV0YXJpYXMvGS9saXN0YS1zZXJ2aWNvcy1vbi1saW5lLzIRL3NlcnZpY29zL2VtcHJlc2ERL3NlcnZpY29zL2NpZGFkYW8cL2xpc3RhLXNpdGVzLWRhLXByZWZlaXR1cmEvNRQrAwZnZ2dnZ2dkZAIEDw9kFgIfBwVJcmV0dXJuIFZhbGlkYXJQZXNxdWlzYSgndHh0UGVzcXVpc2FUb3BvJywnY3RsMDAkbG5rUGVzcXVpc2FUb3BvJywgZXZlbnQpO2QCBg8PFgIeBFRleHQFDlPDoWIsIDEzLzEwLzE4ZGQCBw8PFgIeCEltYWdlVXJsBRV+L2ltYWdlbS90ZW1wby9wYy5naWZkZAIIDw8WAh8JBQQyNcK6ZGQCCQ8PFgIfCQU

In [10]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [11]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, params=params, data=data)
    pages = pages + r.text

In [12]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [13]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [14]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [15]:
print(md5(a_tags[0].text))

43b75d2946867b038517b32edd29fc20


In [16]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [17]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [18]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 5/5 [00:00<00:00, 9957.99it/s]


In [19]:
databases

[{'database_id': '43b75d2946867b038517b32edd29fc20',
  'database_title': 'Unidades Atendimento Ativas Curitiba - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/UnidadesAtendimentoCuritiba/2018-10-09_Unidades_Atendimento_Ativas_Curitiba_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 9)}]

In [26]:
test = pd.read_csv('http://multimidia.curitiba.pr.gov.br/dadosabertos/UnidadesAtendimentoCuritiba/2018-10-09_Unidades_Atendimento_Ativas_Curitiba_-_Base_de_Dados.csv', encoding="UTF-8", delimiter=',', error_bad_lines=False)

In [27]:
test.fillna('').head()

,CD_EQUI,CD_LOCAL,CD_TEMA,DS_TEMA,CD_TP_EQUIPAMENTO,DS_TP_EQUIPAMENTO,CD_SUBTIPO_EQUIPAMENTO,DS_SUBTIPO_EQUIPAMENTO,CD_DEP_ADMINISTRATIVA,DS_DEP_ADMINISTRATIVA,...,LON_SIRGAS,FUNCIONAMENTO_MANHA_EQUI,FUNCIONAMENTO_TARDE_EQUI,FUNCIONAMENTO_NOITE_EQUI,FUNCIONAMENTO_24HRS_EQUI,TELEFONE_EQUI,RAMAL_EQUI,EMAIL_EQUI,SITE_EQUI,FAX_EQUI
0,4.352,,4,PATRIMONIO HISTORICO EDIFICADO,178,Unidade de Interesse Especial de Preservac?o,2,Estilo Art Nouveau,3,Publico Municipal,...,-49,0,0,0,0,,,,,
1,4.353,,4,PATRIMONIO HISTORICO EDIFICADO,27,Unidade de Interesse de Preservac?o,2,Arquitetura Ecletica,11,Particular,...,-49,0,0,0,0,,,,,
2,4.354,,4,PATRIMONIO HISTORICO EDIFICADO,27,Unidade de Interesse de Preservac?o,2,Arquitetura Ecletica,11,Particular,...,-49,0,0,0,0,,,,,
3,4.355,,4,PATRIMONIO HISTORICO EDIFICADO,27,Unidade de Interesse de Preservac?o,2,Arquitetura Ecletica,11,Particular,...,-49,0,0,0,0,,,,,
4,4.357,,4,PATRIMONIO HISTORICO EDIFICADO,27,Unidade de Interesse de Preservac?o,2,Arquitetura Ecletica,11,Particular,...,-49,0,0,0,0,,,,,
